Import modules

In [1]:
from basic_FFN import get_batch_datasets
from json import load
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import matplotlib.pyplot as plt

Load some data

In [4]:
# Get raw datasets.
train_ds = get_batch_datasets(0,
                              800,
                        "agn_{}_synthetic.csv",
                        "/Users/jackhu/PycharmProjects/pytorchSelflearn/data/agn_synthetic")
validate_ds = get_batch_datasets(800,
                                 899,
                             "agn_{}_synthetic.csv",
                             "/Users/jackhu/PycharmProjects/pytorchSelflearn/data/validate")

with open("/Users/jackhu/PycharmProjects/pytorchSelflearn/dataTransformer/trial_configs.json", 'r') as f:
    options = load(f)

# Initialize transformer object, make dataset list a Tensor.
train_ds_tensor = torch.Tensor(np.array(train_ds)).requires_grad_(True)
valid_ds_tensor = torch.Tensor(np.array(validate_ds)).requires_grad_(True)

Getting Datasets from /Users/jackhu/PycharmProjects/pytorchSelflearn/data/agn_synthetic: 100%|██████████| 800/800 [00:01<00:00, 792.74it/s]
Getting Datasets from /Users/jackhu/PycharmProjects/pytorchSelflearn/data/validate: 100%|██████████| 99/99 [00:00<00:00, 717.37it/s]


Positional Encoder

In [2]:
class PositionalEncoding(nn.Module):
    """
    Positional Encoding Module
    """

    def __init__(self, d_model, max_len=5000):
        """
        
        :param d_model: dimension of embedding.
        :param max_len: maximum length of input sequence.
        """
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [3]:
test_pe = PositionalEncoding(32, max_len=100)

In [5]:
test_tensor = train_ds_tensor[0, 1, ...]